In [49]:
import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [50]:
Institutions = "resources/cc_institution_details.csv"
Institutions_db = pd.read_csv(Institutions,encoding="cp1252")

In [51]:
Grads = "resources/cc_institution_grads.csv"
Grads_db = pd.read_csv(Grads)

In [52]:
Institutions_db.head()

,unitid,chronname,city,state,level,control,basic,hbcu,flagship,long_x,...,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,similar,state_sector_ct,carnegie_ct,counted_pct,nicknames,cohort_size
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.568502,...,36.4,5.6,17.2,11.1,232937|100724|405997|113607|139533|144005|2285...,13,386,99.7|07,NaN,882.0
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.809170,...,NaN,NaN,NaN,NaN,196060|180461|201885|145600|209542|236939|1268...,13,106,56.0|07,UAB,1376.0
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,Baccalaureate Colleges--Arts & Sciences,NaN,NaN,-86.174010,...,NaN,NaN,NaN,NaN,217925|441511|205124|247825|197647|221856|1353...,16,252,100.0|07,NaN,3.0
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.638420,...,0.0,0.0,0.0,0.0,232186|133881|196103|196413|207388|171128|1900...,13,106,43.1|07,UAH,759.0
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.295677,...,NaN,NaN,NaN,NaN,100654|232937|242617|243197|144005|241739|2354...,13,386,88.0|07,ASU,1351.0


In [53]:
Inst = Institutions_db[["unitid","chronname","city","state","level","control","student_count","med_sat_value","aid_value","grad_100_value","grad_150_value"]]

In [54]:
Inst

,unitid,chronname,city,state,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,11502,1146.0,6088.0,29.4,53.5
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,322,NaN,2540.0,0.0,66.7
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,5696,1180.0,6647.0,16.5,48.4
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,5356,830.0,7256.0,8.8,25.2
...,...,...,...,...,...,...,...,...,...,...,...
3793,461528,Grace College of Divinity,Fayetteville,North Carolina,4-year,Private not-for-profit,110,NaN,4860.0,0.0,0.0
3794,462354,John Paul the Great Catholic University,Escondido,California,4-year,Private not-for-profit,140,1069.0,9395.0,51.5,51.5
3795,466921,Chamberlain College of Nursing-Missouri,St. Louis,Missouri,4-year,Private for-profit,488,NaN,3291.0,37.5,37.5
3796,474881,Minneapolis Media Institute,Edina,Minnesota,2-year,Private for-profit,147,NaN,3434.0,42.1,55.9


In [55]:
Inst2 = Inst.loc[(Inst["level"] == "4-year")]

In [56]:
len(Inst2)

2339

In [57]:
Inst2 = Inst2.dropna()

In [58]:
len(Inst2)

1297

In [59]:
Howmanystates = Inst2.groupby(["state"]).count()

In [60]:
len(Howmanystates)

51

In [61]:
Grads_db

,unitid,year,gender,race,cohort,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate
0,100760,2011,B,X,2y all,446.0,73.0,105.0,16.4,23.5
1,100760,2011,M,X,2y all,185.0,NaN,40.0,NaN,21.6
2,100760,2011,F,X,2y all,261.0,NaN,65.0,NaN,24.9
3,100760,2011,B,W,2y all,348.0,NaN,86.0,NaN,24.7
4,100760,2011,M,W,2y all,162.0,NaN,35.0,NaN,21.6
...,...,...,...,...,...,...,...,...,...,...
1302097,168591,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN
1302098,168740,2002,F,Ai,4y other,0.0,NaN,0.0,NaN,NaN
1302099,169716,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN
1302100,170082,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN


In [62]:
len(Grads_db)

1302102

In [63]:
#We dropped a lot of records from the Institution database. Now, we have to get rid of all 
#the Institutions we have in Grads data base to have the same amount of schools 
#in the institutions db and the Grads_db.

In [64]:
b=Inst2["unitid"].unique()

In [65]:
len(b)

1297

In [66]:
b2=Grads_db["unitid"].unique()

In [67]:
len(b2)

3979

In [ ]:
#As seen here the amount of Institutions in Inst2 data set is not the same amount of Inst in Grads data set

In [25]:
#This following code creates an array with the schools that are not in InstitutionDB but exsists in Grads_db

In [26]:
StudentswithoutSchools=[]
for index, row in Grads_db.iterrows():
   
    School = row["unitid"]
   # print(School)
    SchoolsinInst = Inst2.loc[(Inst2["unitid"] == School)] 
    InstFlag = SchoolsinInst.empty
    if InstFlag == True:
        if School not in StudentswithoutSchools:
            StudentswithoutSchools.append(School)

In [28]:
len(StudentswithoutSchools)

2682

In [29]:
#The following code creates an array with the schools that are in Institution but not in Grads_db(Just a test) 
#because we expect to get zero

In [30]:
SchoolsWOStudents=[]
for index, row in Inst2.iterrows():
   
    School = row["unitid"]
   # print(School)
    SchoolsinInst = Grads_db.loc[(Grads_db["unitid"] == School)] 
    InstFlag = SchoolsinInst.empty
    if InstFlag == True:
        if School not in SchoolsWOStudents:
            SchoolsWOStudents.append(School)   

In [31]:
#This is expected to be zero because we didn't get rid of anything from Grads db yet
len(SchoolsWOStudents)

0

In [32]:
columns = ["school"]
Table = pd.DataFrame(data=StudentswithoutSchools,columns=columns)

In [38]:
len(Table)

2682

In [34]:
len(Grads_db)

1302102

In [35]:
#This will drop all the students that have no school in the main Institution data.---It dropped 2682 schools that we had null values
#that made 744390 records dropped out of 1302102
for index, row in Table.iterrows():
       
    School = row["school"]
    #print(School)
    indexnumbers = Grads_db[Grads_db['unitid']==School].index
    Grads_db.drop(indexnumbers,inplace=True)

In [ ]:
#Ending with this ammount of students

In [41]:
len(Grads_db)

557712

In [45]:
b=Inst2["unitid"].unique()

In [46]:
len(b)

1297

In [47]:
b2=Grads_db["unitid"].unique()

In [48]:
#As seen here now Grads and Institution data set contain the same amount of Institutions per state.
len(b2)

1297

In [ ]:
#found out we had to get rid of the gender by just grabing all the both in gender and race which are the
#ones that will contain all the info per year and per school or Institution

In [42]:
BothGenders = Grads_db.loc[(Grads_db["gender"] == "B") & (Grads_db["race"] == "B")]

In [43]:
Grads_db2 = BothGenders.groupby(["unitid","year"], as_index=False).sum()

In [44]:
Grads_db2

,unitid,year,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate
0,100654,2002,680.0,139.0,263.0,20.4,38.7
1,100654,2003,748.0,153.0,288.0,20.5,38.5
2,100654,2004,955.0,173.0,314.0,18.1,32.9
3,100654,2005,1046.0,91.0,339.0,8.7,32.4
4,100654,2006,967.0,112.0,326.0,11.6,33.7
...,...,...,...,...,...,...,...
15487,454184,2012,1.0,0.0,0.0,0.0,0.0
15488,454184,2013,1.0,1.0,1.0,100.0,100.0
15489,462354,2011,0.0,0.0,0.0,0.0,0.0
15490,462354,2012,1.0,0.0,0.0,0.0,0.0


In [402]:
Grads_db2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15492 entries, 0 to 15491
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   unitid         15492 non-null  int64  
 1   year           15492 non-null  int64  
 2   grad_cohort    15492 non-null  float64
 3   grad_100       15492 non-null  float64
 4   grad_150       15492 non-null  float64
 5   grad_100_rate  15492 non-null  float64
 6   grad_150_rate  15492 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 847.3 KB


In [436]:
#IndexG = Grads_db2.loc[(Grads_db2["grad_cohort"] == 0) & (Grads_db2["grad_100"] == 0) & (Grads_db2["grad_150"] == 0)].index

In [438]:
#Grads_db2.drop(IndexG,inplace=True)

In [480]:
Grads_db2.to_csv('resources/Grads_db2.csv')

In [481]:
Inst2.to_csv('resources/Inst2.csv')